In [21]:
%%writefile /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py

from tqdm.autonotebook import tqdm
import time
import pandas as pd
import numpy as np
import os
from random import randint, shuffle
from skimage import io
from matplotlib import pyplot as plt
import seaborn
import logging

from sklearn.model_selection import train_test_split, StratifiedKFold, RepeatedStratifiedKFold
# from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import *

import optuna

# from torchvision.models import vgg16, VGG16_Weights
from torchvision.models import *
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn
import torch


def binary_labeling_pos_neg(x):
    if x.split('/')[7] == 'MODS_Positivas':
        return 1
    elif x.split('/')[7] == 'MODS_Negativas':
        return 0

def binary_labelling_7_9(x):
    if x.split('/')[7] == 'MODS_Positivas':
        if x.split('/')[-3].split('_')[1] == '7':
            return 0
        elif x.split('/')[-3].split('_')[1] == '9':
            return 1
    elif x.split('/')[7] == 'MODS_Negativas':
        return np.nan
    
def multiclass_labeling_neg_7_9(x):
    if x.split('/')[7] == 'MODS_Positivas':
        if x.split('/')[-3].split('_')[1] == '7':
            return 1
        elif x.split('/')[-3].split('_')[1] == '9':
            return 2
    elif x.split('/')[7] == 'MODS_Negativas':
        return 0


def get_dataframe(path, mode='binary_pos_neg'):
    
    path_images = []

    for root, dirs, files in os.walk(path):
        for file in files:
            path_images.append(os.path.join(root, file))

    df_data = pd.DataFrame({'path_images': path_images})
    
    df_data['img_name'] = df_data['path_images'].apply(lambda x: x.split('/')[-1])
    
    if mode == 'binary_pos_neg':
        df_data['label'] = df_data['path_images'].apply(binary_labeling_pos_neg)
    elif mode == 'multiclass_neg_7_9':
        df_data['label'] = df_data['path_images'].apply(multiclass_labeling_neg_7_9)
    elif mode == 'binary_7_9':
        df_data['label'] = df_data['path_images'].apply(binary_labelling_7_9)
        df_data.dropna(subset=['label'], inplace=True)
        df_data.reset_index(drop=True, inplace=True)
        df_data['label'] = df_data['label'].astype('int32')
    
    return df_data

def split_dataframe(df_data, test_size=0.2, random_state=None):
    
    df_cross_val, df_test = train_test_split(df_data, 
                                             test_size=test_size, 
                                             random_state=random_state, 
                                             stratify=df_data['label'])

    df_cross_val.reset_index(drop=True, inplace=True)
    df_test.reset_index(drop=True, inplace=True)
    
    return df_cross_val, df_test

Overwriting /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py


In [22]:
%%writefile -a /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py


class Dataset_MODS(Dataset):
    def __init__(self, df, transform=None):
        self.df_data = df
        self.transform = transform

    def __getitem__(self, idx):
        path_img = self.df_data['path_images'][idx]
        label = self.df_data['label'][idx]
        img = io.imread(path_img)

        if self.transform:
            img = self.transform(img.copy())
        
        return img, label

    def __len__(self):
        return self.df_data.shape[0]

Appending to /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py


In [23]:
%%writefile -a /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py

def moveTo(obj, device):

    if isinstance(obj, list):
        return [moveTo(x, device) for x in obj]
    elif isinstance(obj, tuple):
        return tuple(moveTo(list(obj), device))
    elif isinstance(obj, set):
        return set(moveTo(list(obj), device))
    elif isinstance(obj, dict):
        to_ret = dict()
        for key, value in obj.items():
            to_ret[moveTo(key, device)] = moveTo(value, device)
        return to_ret
    elif hasattr(obj, "to"):
        return obj.to(device)
    else:
        return obj

Appending to /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py


In [24]:
%%writefile -a /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py

def run_epoch(model, optimizer, data_loader, loss_func, device, results, score_funcs, prefix="", desc=None):
    
    running_loss = []
    y_true = []
    y_pred = []
    
    start = time.time()
    
    for inputs, labels in tqdm(data_loader, desc=desc, leave=False):
        
        labels = labels.type(torch.LongTensor)
        
        inputs = moveTo(inputs, device)
        labels = moveTo(labels, device)

        y_hat = model(inputs)
        loss = loss_func(y_hat, labels)

        if model.training:
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        running_loss.append(loss.item())

        if len(score_funcs) > 0 and isinstance(labels, torch.Tensor):
            labels = labels.detach().cpu().numpy()
            y_hat = y_hat.detach().cpu().numpy()
            y_true.extend(labels.tolist())
            y_pred.extend(y_hat.tolist())
            
    end = time.time()
    
    y_pred = np.asarray(y_pred)
    
    if len(y_pred.shape) == 2 and y_pred.shape[1] > 1: 
        y_pred = np.argmax(y_pred, axis=1)
    
    results[prefix + " loss"].append(np.mean(running_loss))

    for name, score_func in score_funcs.items():
        try:
            if name == 'Accuracy':
                results[prefix + " " + name].append(score_func(y_true, y_pred))
            elif name == 'Precision':
                results[prefix + " " + name].append(score_func(y_true, y_pred, pos_label=1))
            elif name == 'Sensitivity':
                results[prefix + " " + name].append(score_func(y_true, y_pred, pos_label=1))
            elif name == 'Specificity':
                results[prefix + " " + name].append(score_func(y_true, y_pred, pos_label=0))
            elif name == 'F1_score':
                results[prefix + " " + name].append(score_func(y_true, y_pred, pos_label=1))
            elif name == 'ROC AUC Score':
                results[prefix + " " + name].append(score_func(y_true, y_pred))
            elif name == 'Quadratic Weighted Kappa':
                results[prefix + " " + name].append(score_func(y_true, y_pred, weights='quadratic')) 
        except:
            results[prefix + " " + name].append(float("NaN"))
            
    return end-start

Appending to /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py


In [25]:
%%writefile -a /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py

def train_model(model, loss_func, train_loader, val_loader=None, test_loader=None, score_funcs=None, 
                epochs=10, device="cpu", checkpoint_file=None, lr_schedule=None, optimizer=None, disable_tqdm=False):
    
    if score_funcs == None:
        score_funcs = {}
    
    to_track = ["epoch", "total time", "train loss"]
    
    if val_loader is not None:
        to_track.append("val loss")
    if test_loader is not None:
        to_track.append("test loss")
        
    for eval_score in score_funcs:
        to_track.append("train " + eval_score )
        
        if val_loader is not None:
            to_track.append("val " + eval_score )
        if test_loader is not None:
            to_track.append("test "+ eval_score )
    
        
    total_train_time = 0
    results = {}
    
    for item in to_track:
        results[item] = []

    if optimizer == None:
        optimizer = torch.optim.AdamW(model.parameters())

    model.to(device)
    
    for epoch in tqdm(range(epochs), desc="Epoch", disable=disable_tqdm):
        
        model = model.train()

        total_train_time += run_epoch(model, optimizer, train_loader, loss_func, device, results, score_funcs, prefix="train", desc="Training")
        
        results["epoch"].append(epoch)
        results["total time"].append(total_train_time)
        
      
        if val_loader is not None:
            model = model.eval()
            with torch.no_grad():
                run_epoch(model, optimizer, val_loader, loss_func, device, results, score_funcs, prefix="val", desc="Validating")
                
        if lr_schedule is not None:
            if isinstance(lr_schedule, torch.optim.lr_scheduler.ReduceLROnPlateau):
                lr_schedule.step(results["val loss"][-1])
            else:
                lr_schedule.step()
                
        if test_loader is not None:
            model = model.eval()
            with torch.no_grad():
                run_epoch(model, optimizer, test_loader, loss_func, device, results, score_funcs, prefix="test", desc="Testing")
        
        if checkpoint_file is not None:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'results' : results
                }, checkpoint_file)

    return pd.DataFrame.from_dict(results)

Appending to /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py


In [26]:
%%writefile -a /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py

def get_base_model(name_model='vgg16'):
    
    if name_model == 'vgg16':
        weights = VGG16_Weights.DEFAULT
        model = vgg16(weights=weights)
        in_features = model.classifier[0].in_features
    elif name_model == 'vgg16_bn':
        weights = VGG16_BN_Weights.DEFAULT
        model = vgg16_bn(weights=weights)
        in_features = model.classifier[0].in_features
        
    return model, weights, in_features

def get_optimizer(model, lr, name_optimizer):
    
    if name_optimizer == 'AdamW':
        optimizer = torch.optim.AdamW(model.parameters(), lr)
        
    return optimizer

def get_loss_func(name_loss_func):
    
    if name_loss_func == 'CrossEntropyLoss':
        loss_func = nn.CrossEntropyLoss()
    
    return loss_func

def get_metric(name_metric='Accuracy'):
    
    if name_metric == 'Accuracy':
        score_funcs = {name_metric: accuracy_score}
    elif name_metric == 'Quadratic Weighted Kappa':
        score_funcs = {name_metric: cohen_kappa_score}
    
    return score_funcs

Appending to /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py


In [27]:
%%writefile -a /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py

class NormalizeInput(nn.Module):
    def __init__(self, baseModel, weights):
        super(NormalizeInput, self).__init__()
        self.baseModel = baseModel
        self.preprocess = weights.transforms()
    
    def forward(self, input):
        
        input = self.preprocess(input)
        
        return self.baseModel(input)

Appending to /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py


In [28]:
%%writefile -a /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py

def get_hyperparameters(path, idx):
    
    df_hyper = pd.read_csv(os.path.join(path, 'hyperparameters.txt'), sep='\t')
    dict_hyper = {}
    
    ## Epochs
    
    df_n_epochs = pd.read_csv(os.path.join(path, 'n_epochs.txt'), sep='\t')
    
    dict_hyper['n_epochs_min'] = int(df_n_epochs['n_epochs_min'][0])
    dict_hyper['n_epochs_max'] = int(df_n_epochs['n_epochs_max'][0])
    dict_hyper['n_epochs_step'] = int(df_n_epochs['n_epochs_step'][0])
    dict_hyper['n_epochs_range'] = eval(df_n_epochs['n_epochs_range'][0])
    
    if np.isnan(df_hyper['n_epochs'][idx]):
        dict_hyper['n_epochs'] = np.nan
    else:
        dict_hyper['n_epochs'] = int(df_hyper['n_epochs'][idx])
    
    
    ## Batch size
    
    df_batch_size = pd.read_csv(os.path.join(path, 'batch_size.txt'), sep='\t')

    dict_hyper['batch_size_min'] = int(df_batch_size['batch_size_min'][0])
    dict_hyper['batch_size_max'] = int(df_batch_size['batch_size_max'][0])
    dict_hyper['batch_size_step'] = int(df_batch_size['batch_size_step'][0])
    dict_hyper['batch_size_range'] = eval(df_batch_size['batch_size_range'][0])
    
    if np.isnan(df_hyper['batch_size'][idx]):
        dict_hyper['batch_size'] = np.nan
    else:
        dict_hyper['batch_size'] = int(df_hyper['batch_size'][idx])
    
    
    ## Hidden layers
    
    df_hidden_layers = pd.read_csv(os.path.join(path, 'hidden_layers.txt'), sep='\t')

    dict_hyper['hidden_layers_min'] = int(df_hidden_layers['hidden_layers_min'][0])
    dict_hyper['hidden_layers_max'] = int(df_hidden_layers['hidden_layers_max'][0])
    dict_hyper['hidden_layers_step'] = int(df_hidden_layers['hidden_layers_step'][0])
    dict_hyper['hidden_layers_range'] = eval(df_hidden_layers['hidden_layers_range'][0])

    if np.isnan(df_hyper['hidden_layers'][idx]):
        dict_hyper['hidden_layers'] = np.nan
    else:
        dict_hyper['hidden_layers'] = int(df_hyper['hidden_layers'][idx])
    
    
    ## Neurons per layer
    
    df_neurons_per_layer = pd.read_csv(os.path.join(path, 'neurons_per_layer.txt'), sep='\t')
    
    dict_hyper['neurons_per_layer_min'] = int(df_neurons_per_layer['neurons_per_layer_min'][0])
    dict_hyper['neurons_per_layer_max'] = int(df_neurons_per_layer['neurons_per_layer_max'][0])
    dict_hyper['neurons_per_layer_step'] = int(df_neurons_per_layer['neurons_per_layer_step'][0])
    dict_hyper['neurons_per_layer_range'] = eval(df_neurons_per_layer['neurons_per_layer_range'][0])

    if np.isnan(df_hyper['neurons_per_layer'][idx]):
        dict_hyper['neurons_per_layer'] = np.nan
    else:
        dict_hyper['neurons_per_layer'] = int(df_hyper['neurons_per_layer'][idx])
    
    
    ## Learning rate
    
    df_learning_rate = pd.read_csv(os.path.join(path, 'learning_rate.txt'), sep='\t')
    
    dict_hyper['learning_rate_min'] = float(df_learning_rate['learning_rate_min'][0])
    dict_hyper['learning_rate_max'] = float(df_learning_rate['learning_rate_max'][0])
    dict_hyper['learning_rate_range'] = eval(df_learning_rate['learning_rate_range'][0])
    
    if np.isnan(df_hyper['learning_rate'][idx]):
        dict_hyper['learning_rate'] = np.nan
    else:
        dict_hyper['learning_rate'] = float(df_hyper['learning_rate'][idx])
    
    
    ## Other Hyperparameters
    
    dict_hyper['for_optimization'] = str(df_hyper['for_optimization'][idx])
    
    if np.isnan(df_hyper['test_size'][idx]):
        dict_hyper['test_size'] = 0.2
    else:
        dict_hyper['test_size'] = float(df_hyper['test_size'][idx])
        
    if type(df_hyper['random_state'][idx]) == str:
        dict_hyper['random_state'] = None
    else:
        dict_hyper['random_state'] = int(df_hyper['random_state'][idx])    
        
    if np.isnan(df_hyper['val_size'][idx]):
        dict_hyper['val_size'] = 0.25
    else:
        dict_hyper['val_size'] = float(df_hyper['val_size'][idx])
    
    if type(df_hyper['mode'][idx]) == str:
        dict_hyper['mode'] = str(df_hyper['mode'][idx])
    else:
        dict_hyper['mode'] = 'binary_pos_neg'
    
    if type(df_hyper['base_model'][idx]) == str:
        dict_hyper['base_model'] = str(df_hyper['base_model'][idx])
    else:
        dict_hyper['base_model'] = 'vgg16'
    
    if type(df_hyper['optimizer'][idx]) == str:
        dict_hyper['optimizer'] = str(df_hyper['optimizer'][idx])
    else:
        dict_hyper['optimizer'] = 'AdamW'
        
    if type(df_hyper['loss_func'][idx]) == str:
        dict_hyper['loss_func'] = str(df_hyper['loss_func'][idx])
    else:
        dict_hyper['loss_func'] = 'CrossEntropyLoss'
        
    if type(df_hyper['metric'][idx]) == str:
        dict_hyper['metric'] = str(df_hyper['metric'][idx])
    else:
        dict_hyper['metric'] = 'Accuracy'
        
    if type(df_hyper['dataset'][idx]) == str:
        dict_hyper['dataset'] = str(df_hyper['dataset'][idx])
    else:
        dict_hyper['dataset'] = 'val'
        
    if type(df_hyper['direction'][idx]) == str:
        dict_hyper['direction'] = str(df_hyper['direction'][idx])
    else:
        dict_hyper['direction'] = 'maximize'
        
    if np.isnan(df_hyper['n_trials'][idx]):
        dict_hyper['n_trials'] = np.nan
    else:
        dict_hyper['n_trials'] = int(df_hyper['n_trials'][idx])
        
    if np.isnan(df_hyper['n_jobs'][idx]):
        dict_hyper['n_jobs'] = np.nan
    else:
        dict_hyper['n_jobs'] = int(df_hyper['n_jobs'][idx])
        
    return df_hyper, dict_hyper

def get_time(start_time, end_time):
    
    total_time = end_time-start_time
    
    if total_time <= 60:
        return str(total_time) + ' ' + 'sec'
    elif 60 < total_time <= 3600:
        return str(round(total_time/60, 2)) + ' ' + 'min'
    elif 3600 < total_time <= 86400:
        return str(round(total_time/3600, 2)) + ' ' + 'hours'
    elif total_time > 86400:
        return str(round(total_time/86400, 2)) + ' ' + 'days'
    
def save_logfile(path, total_results, test_results, used_hyperparams, other_hyperparams, idx, id_trial):
    

    logging.basicConfig(level=logging.INFO, format = '%(levelname)s - %(message)s', 
                        filename=os.path.join(path, 'logfiles_crossval_train_test','logfile_crossval_train_test_' + str(idx) + '_' + str(id_trial) + '.log'))
    
    logging.info('########################################################################################')
    logging.info('CROSS-VALIDATION')
    logging.info('HYPERPARAMETERS - batch_size: %d - n_epochs: %d - hidden_layers: %d - neurons_per_layer: %d - learning_rate: %f - n_folds: %d - n_iters: %d' % used_hyperparams)
    logging.info('OTHER HYPERPARAMETERS - mode: %s - base_model: %s - optimizer: %s - loss_func: %s - test_size: %f' % other_hyperparams)
    
    if other_hyperparams[0].split('_')[0] == 'binary':
        if other_hyperparams[0].split('_')[1] == 'pos':
            logging.info('Class 0: Negative')
            logging.info('Class 1: Positive')
        elif other_hyperparams[0].split('_')[1] == '7':
            logging.info('Class 0: 7 days')
            logging.info('Class 1: 9 days') 
    elif other_hyperparams[0].split('_')[0] == 'multiclass':
        logging.info('Class 0: Negative')
        logging.info('Class 1: 7 days')
        logging.info('Class 2: 9 days')
        
    
    for i in range(total_results.shape[0]):
        string_row = []
        for name_col in total_results.columns:
            string_row.append(str(name_col) + ': ' + str(total_results[name_col][i]))

        logging.info(' - '.join(string_row))
            
    logging.info('########################################################################################')
    logging.info('TRAINING-TESTING')
    logging.info('HYPERPARAMETERS - batch_size: %d - n_epochs: %d - hidden_layers: %d - neurons_per_layer: %d - learning_rate: %f - n_folds: %d - n_iters: %d' % used_hyperparams)
    logging.info('OTHER HYPERPARAMETERS - mode: %s - base_model: %s - optimizer: %s - loss_func: %s - test_size: %f' % other_hyperparams)
    
    if other_hyperparams[0].split('_')[0] == 'binary':
        if other_hyperparams[0].split('_')[1] == 'pos':
            logging.info('Class 0: Negative')
            logging.info('Class 1: Positive')
        elif other_hyperparams[0].split('_')[1] == '7':
            logging.info('Class 0: 7 days')
            logging.info('Class 1: 9 days') 
    elif other_hyperparams[0].split('_')[0] == 'multiclass':
        logging.info('Class 0: Negative')
        logging.info('Class 1: 7 days')
        logging.info('Class 2: 9 days')
    
    for i in range(test_results.shape[0]):
        string_row = []
        for name_col in test_results.columns:
            string_row.append(str(name_col) + ': ' + str(test_results[name_col][i]))

        logging.info(' - '.join(string_row))

    logging.shutdown()

Appending to /tf/mnt/mario/projects/mods_v1/scripts/module_mods3d.py


In [29]:
%%writefile /tf/mnt/mario/projects/mods_v1/scripts/hyper_optm_mods3d.py

import logging
import argparse
from module_mods3d import *

# path_project = '/tf/mnt/mario/projects/mods_v1'
# path_dataset = '/tf/mnt/mario/projects/mods_v1/dataset'

parser = argparse.ArgumentParser(description='Load index and path for hyperparameter optimization')
parser.add_argument('-idx', type=int, help='Index of the set of hyperparameters')
parser.add_argument('-path', type=str, help='Path of the project')
args = parser.parse_args()

idx = args.idx
path_project = args.path


df_hyper, dict_hyper = get_hyperparameters(os.path.join(path_project, 'optim_hyperparams'), idx=idx)



## Epochs info

n_epochs_min = dict_hyper['n_epochs_min']
n_epochs_max = dict_hyper['n_epochs_max']
n_epochs_step = dict_hyper['n_epochs_step']
n_epochs_range = dict_hyper['n_epochs_range']


## Batch size info

batch_size_min = dict_hyper['batch_size_min']
batch_size_max = dict_hyper['batch_size_max']
batch_size_step = dict_hyper['batch_size_step']
batch_size_range = dict_hyper['batch_size_range']


## Hidden layers info

hidden_layers_min = dict_hyper['hidden_layers_min']
hidden_layers_max = dict_hyper['hidden_layers_max']
hidden_layers_step = dict_hyper['hidden_layers_step']
hidden_layers_range = dict_hyper['hidden_layers_range']


## Neurons per layer info

neurons_per_layer_min = dict_hyper['neurons_per_layer_min']
neurons_per_layer_max = dict_hyper['neurons_per_layer_max']
neurons_per_layer_step = dict_hyper['neurons_per_layer_step']
neurons_per_layer_range = dict_hyper['neurons_per_layer_range']


## Learning rate info

learning_rate_min = dict_hyper['learning_rate_min']
learning_rate_max = dict_hyper['learning_rate_max']
learning_rate_range = dict_hyper['learning_rate_range']


## Other hyperparameters

hyper_params_for_optim = dict_hyper['for_optimization']
mode = dict_hyper['mode']

test_size = dict_hyper['test_size']
val_size = dict_hyper['val_size']
random_state = dict_hyper['random_state']

direction = dict_hyper['direction']
n_trials = dict_hyper['n_trials']
n_jobs = dict_hyper['n_jobs']

name_base_model = dict_hyper['base_model']
name_optimizer = dict_hyper['optimizer']
name_loss_func = dict_hyper['loss_func']
name_metric = dict_hyper['metric']
name_dataset = dict_hyper['dataset']

n_epochs = dict_hyper['n_epochs']
batch_size = dict_hyper['batch_size']
hidden_layers = dict_hyper['hidden_layers']
neurons_per_layer = dict_hyper['neurons_per_layer']
learning_rate = dict_hyper['learning_rate']
                       
score_funcs = get_metric(name_metric=name_metric)


## Getting dataset

df_data = get_dataframe(os.path.join(path_project, 'dataset'), mode=mode)
df_cross_val, df_test = split_dataframe(df_data, test_size=test_size, random_state=random_state)

cross_val_dataset = Dataset_MODS(df_cross_val, transform=transforms.ToTensor())
test_dataset = Dataset_MODS(df_test, transform=transforms.ToTensor())


## Setting up device

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


## Optimization objective function

def objective(trial):
    
    train_idx, val_idx = train_test_split(np.arange(len(cross_val_dataset)),
                                          test_size=val_size,
                                          shuffle=True,
                                          stratify=cross_val_dataset.df_data.label)
    
    train_dataset = Subset(cross_val_dataset, train_idx)
    val_dataset = Subset(cross_val_dataset, val_idx)
    
    if np.isnan(batch_size):
        # bs = trial.suggest_int(name='batch_size', low=8, high=32, step=8)
        bs = trial.suggest_int(name='batch_size', 
                               low=batch_size_min, 
                               high=batch_size_max, 
                               step=batch_size_step)
    else:
        bs = batch_size
    
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=bs, shuffle=False)
    
    # Hidden layer size
    if np.isnan(neurons_per_layer):
        # neu_per_lay = trial.suggest_int(name='neurons_per_layer', low=16, high=512, step=16)
        neu_per_lay = trial.suggest_int(name='neurons_per_layer', 
                                        low=neurons_per_layer_min, 
                                        high=neurons_per_layer_max, 
                                        step=neurons_per_layer_step)
    else:
        neu_per_lay = neurons_per_layer
    
    if np.isnan(hidden_layers):
        # hidd_lay = trial.suggest_int(name='hidden_layers', low=1, high=4, step=1)
        hidd_lay = trial.suggest_int(name='hidden_layers', 
                                     low=hidden_layers_min, 
                                     high=hidden_layers_max, 
                                     step=hidden_layers_step)
    else:
        hidd_lay = hidden_layers
    
    model, weights, in_features = get_base_model(name_model=name_base_model)

    for param in model.parameters():
        param.requires_grad = False

    classes = df_data['label'].unique().shape[0]

    classifier_layers = [nn.Flatten(),
                         nn.Linear(in_features, neu_per_lay),
                         nn.ReLU()
                        ]

    for _ in range(hidd_lay-1):
        classifier_layers.append(nn.Linear(neu_per_lay, neu_per_lay))
        classifier_layers.append(nn.ReLU())
    
    classifier_layers.append(nn.Linear(neu_per_lay, classes))
     
    fc_classifier_model = nn.Sequential(*classifier_layers)
    
    model.classifier = fc_classifier_model
    
    model = NormalizeInput(model, weights)
    
    if np.isnan(learning_rate):
        # lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
        lr = trial.suggest_float(name='learning_rate', 
                                 low=learning_rate_min, 
                                 high=learning_rate_max, 
                                 log=True)
    else:
        lr = learning_rate

    optimizer = get_optimizer(model=model, lr=lr, name_optimizer=name_optimizer)
    
    loss_func = get_loss_func(name_loss_func=name_loss_func)
    
    if np.isnan(n_epochs):
        # n_epoc = trial.suggest_int(name='n_epochs', low=10, high=100, step=10)
        n_epoc = trial.suggest_int(name='n_epochs', 
                                   low=n_epochs_min, 
                                   high=n_epochs_max, 
                                   step=n_epochs_step)

    else:
        n_epoc = n_epochs
        
    results = train_model(model, loss_func, train_loader, val_loader=val_loader, score_funcs=score_funcs, 
                          epochs=n_epoc, device=device, optimizer=optimizer, disable_tqdm=True)
    
    return results[name_dataset + ' ' + name_metric].iloc[-1]


## Optimization procedure

def optimization():
                          
    search_space = {}
                          
    if np.isnan(batch_size):
        # search_space['batch_size'] = [8, 16, 32]
        search_space['batch_size'] = batch_size_range
    if np.isnan(neurons_per_layer):
        # search_space['neurons_per_layer'] = [16, 32, 64, 128, 256, 512]
        search_space['neurons_per_layer'] = neurons_per_layer_range
    if np.isnan(hidden_layers):
        # search_space['hidden_layers'] = [1, 2, 3, 4]
        search_space['hidden_layers'] = hidden_layers_range
    if np.isnan(n_epochs):
        # search_space['n_epochs'] = [10, 30, 50, 80, 100]
        search_space['n_epochs'] = n_epochs_range
    if np.isnan(learning_rate):
        # search_space['learning_rate'] = [1e-5, 1e-4, 1e-3, 1e-2]
        search_space['learning_rate'] = learning_rate_range
    
    
#     search_space = {'batch_size': [8, 16, 32, 64], 
#                     'neurons_per_layer': [16, 32, 64, 128, 256, 512], 
#                     'hidden_layers': [1, 2, 3, 4], 
#                     'n_epochs': [10, 30, 50, 80, 100],
#                     'learning_rate': [1e-5, 1e-4, 1e-3, 1e-2]}
              
    ### To save logging info of the optmization by trial
    # filename = os.path.join(path_project, 'log_files_optimization', 'logfile_optim_' + str(idx) + '.log')
    # optuna.logging.get_logger('optuna').addHandler(logging.FileHandler(filename=filename, mode='w'))
    
    sampler = optuna.samplers.GridSampler(search_space)
    study_name = 'HYPERPARAMETER OPTIMIZATION WITH OPTUNA: ' + \
                  mode + ' - ' + name_metric + ' - ' + hyper_params_for_optim + ' - ' + str(search_space)
    
    study = optuna.create_study(study_name=study_name, 
                                direction=direction, 
                                sampler=sampler)
    
    study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs)
    
    return study
    

if __name__ == '__main__':
    
    start_time = time.time()
    
    print()
    print('START OPTIMIZATION - Optimization of:', hyper_params_for_optim)
    print()
    
    study = optimization()
    
    end_time = time.time()
    
    print()
    print('END OPTIMIZATION')
    print()
    
    total_time = get_time(start_time, end_time)
    
    
    if np.isnan(n_epochs):
        n_epochs = int(study.best_params['n_epochs'])
        df_hyper.loc[idx, 'n_epochs'] = str(n_epochs)
    if np.isnan(batch_size):
        batch_size = int(study.best_params['batch_size'])
        df_hyper.loc[idx, 'batch_size'] = str(batch_size)
    if np.isnan(hidden_layers):
        hidden_layers = int(study.best_params['hidden_layers'])
        df_hyper.loc[idx, 'hidden_layers'] = str(hidden_layers)
    if np.isnan(neurons_per_layer):
        neurons_per_layer = int(study.best_params['neurons_per_layer'])
        df_hyper.loc[idx, 'neurons_per_layer'] = str(neurons_per_layer)
    if np.isnan(learning_rate):
        learning_rate = float(study.best_params['learning_rate'])
        df_hyper.loc[idx, 'learning_rate'] = str(learning_rate)

    best_value = study.best_value
    
    df_hyper.loc[idx, 'best_value'] = best_value
    df_hyper.loc[idx, 'time'] = str(total_time)
    df_hyper.loc[idx, 'best_trial_number'] = str(study.best_trial.number)
    
    ### To save optimized hyperparameters
    # df_hyper.to_csv(os.path.join(path_project, 'optim_hyperparams', 'hyperparameters.txt'), index=False, sep='\t', mode='w+')
    
    print(study.best_params)
    print('Best Value ' + '(' + name_dataset + ' ' + name_metric + '):', study.best_value)
    print('Best trial number:', study.best_trial.number)
    print('Total time:' + total_time)
    print()

Overwriting /tf/mnt/mario/projects/mods_v1/scripts/hyper_optm_mods3d.py


In [30]:
%%writefile /tf/mnt/mario/projects/mods_v1/scripts/crossval_train_mods3d.py


import logging
import argparse
from module_mods3d import *

# path_project = '/tf/mnt/mario/projects/mods_v1'

parser = argparse.ArgumentParser(description='Load info for cross validation training')

parser.add_argument('-idx', type=int, help='Index of the set of hyperparameters')
parser.add_argument('-path', type=str, help='Path of the project')
parser.add_argument('-batch_size', type=int, help='Batch size')
parser.add_argument('-n_epochs', type=int, help='Number of epochs')
parser.add_argument('-hidden_layers', type=int, help='Number of hidden layers')
parser.add_argument('-neurons_per_layer', type=int, help='Number of neurons per layer')
parser.add_argument('-learning_rate', type=float, help='Learning rate value')
parser.add_argument('-test_size_crossval_train', type=float, help='Test size for cross-validation and training experimentation')
parser.add_argument('-n_folds', type=int, help='Number of k-fold cross validation')
parser.add_argument('-n_iters', type=int, help='Number of iterations')
parser.add_argument('-id_trial', type=int, help='Number of trial for different configurations in optimization')

args = parser.parse_args()

idx = args.idx
path_project = args.path
# run_id = args.run_id
id_trial = args.id_trial


df_hyper, dict_hyper = get_hyperparameters(os.path.join(path_project, 'optim_hyperparams'), idx=idx)


## HYPERPARAMETERS

batch_size = args.batch_size
n_epochs = args.n_epochs
hidden_layers = args.hidden_layers
neurons_per_layer = args.neurons_per_layer
learning_rate = args.learning_rate



## OTHER HYPERPARAMETERS

mode = dict_hyper['mode']
# mode = 'binary_pos_neg'

test_size = args.test_size_crossval_train
# test_size = dict_hyper['test_size']
# test_size = 0.2

# val_size = dict_hyper['val_size']
# val_size = 0.25


random_state = dict_hyper['random_state']
# random_state = None

name_base_model = dict_hyper['base_model']
name_optimizer = dict_hyper['optimizer']
name_loss_func = dict_hyper['loss_func']
name_metric = dict_hyper['metric']
name_dataset = dict_hyper['dataset']

## Getting dataset

df_data = get_dataframe(os.path.join(path_project, 'dataset'), mode=mode)


## Setting up device

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


n_folds = args.n_folds
# n_folds = 4

n_iters = args.n_iters
# n_iters = 2


## METRICS

if mode.split('_')[0] == 'binary':
    score_funcs = {'Accuracy': accuracy_score,
                   'Precision': precision_score,
                   'Sensitivity': recall_score,
                   'Specificity': recall_score,
                   'F1_score': f1_score,
                   'ROC AUC Score': roc_auc_score,
                   'Quadratic Weighted Kappa': cohen_kappa_score}
    
elif mode.split('_')[0] == 'multiclass':
    score_funcs = {'Accuracy': accuracy_score}
    


if __name__ == '__main__':
    
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=None)

    ## Defining Transfer learning model

    model, weights, in_features = get_base_model(name_model=name_base_model)

    for param in model.parameters():
        param.requires_grad = False

    classes = df_data['label'].unique().shape[0]

    classifier_layers = [nn.Flatten(),
                         nn.Linear(in_features, neurons_per_layer),
                         nn.ReLU()]

    for _ in range(hidden_layers-1):
        classifier_layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
        classifier_layers.append(nn.ReLU())

    classifier_layers.append(nn.Linear(neurons_per_layer, classes)) 
    fc_classifier_model = nn.Sequential(*classifier_layers)

    model.classifier = fc_classifier_model
    model = NormalizeInput(model, weights)

    optimizer = get_optimizer(model=model, lr=learning_rate, name_optimizer=name_optimizer)
    loss_func = get_loss_func(name_loss_func=name_loss_func)


    ## Training K-fold Cross-Validation

    cross_val_results = pd.DataFrame()
    test_results = pd.DataFrame()

    for i in range(n_iters):

        df_cross_val, df_test = split_dataframe(df_data, test_size=test_size, random_state=random_state)

        cross_val_dataset = Dataset_MODS(df_cross_val, transform=transforms.ToTensor())
        test_dataset = Dataset_MODS(df_test, transform=transforms.ToTensor())
        
        cross_val_loader = DataLoader(cross_val_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        print()
        print('ITERATION:', i+1)

        for k, (train_idxs, val_idxs) in enumerate(skf.split(X=np.arange(len(cross_val_dataset)), 
                                                             y=cross_val_dataset.df_data['label'])):
            
            print('Fold', k+1)

            train_dataset = Subset(cross_val_dataset, train_idxs)
            val_dataset = Subset(cross_val_dataset, val_idxs)

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

            results = train_model(model, loss_func, train_loader, test_loader=test_loader, val_loader=val_loader, 
                                  score_funcs=score_funcs, epochs=n_epochs, device=device, optimizer=optimizer, 
                                  disable_tqdm=True)

            results.insert(0, 'k_fold', n_epochs*[k+1])
            results.insert(0, 'N_iteration', n_epochs*[i+1])
    

            if cross_val_results.empty:
                cross_val_results = results.copy()
            else:
                cross_val_results = pd.concat([cross_val_results, results], ignore_index=True)

            for layer in model.baseModel.classifier:
                if hasattr(layer, 'reset_parameters'):
                    layer.reset_parameters() 
        
        print('Training and testing on iteration', i+1)
        
        results = train_model(model, loss_func, cross_val_loader, test_loader=test_loader, score_funcs=score_funcs, 
                              epochs=n_epochs, device=device, optimizer=optimizer, disable_tqdm=True)
        
        results.insert(0, 'N_iteration', n_epochs*[i+1])
        
        if test_results.empty:
            test_results = results.copy()
        else:
            test_results = pd.concat([test_results, results], ignore_index=True)
            
        for layer in model.baseModel.classifier:
                if hasattr(layer, 'reset_parameters'):
                    layer.reset_parameters()
            
        print()
        
        
    cross_val_results.to_csv(os.path.join(path_project, 'csvfiles_crossval','csvfile_crossval_' + str(idx) + '_' + str(id_trial) + '.csv'), index=False)
    test_results.to_csv(os.path.join(path_project, 'csvfiles_test','csvfile_test_' + str(idx) + '_'+ str(id_trial) + '.csv'), index=False)
    
    used_hyperparams = (batch_size, n_epochs, hidden_layers, neurons_per_layer, learning_rate, n_folds, n_iters)
    other_hyperparams = (mode, name_base_model, name_optimizer, name_loss_func, test_size)
    save_logfile(path_project, cross_val_results, test_results, used_hyperparams, other_hyperparams, idx, id_trial)

Overwriting /tf/mnt/mario/projects/mods_v1/scripts/crossval_train_mods3d.py
